In [6]:
import requests
f=open("authors.txt","r",encoding="utf-8")
url="https://api.openalex.org/works?filter=author.id:"
author_ids=[]
work_count=0
count=0
for line in f:
    temp=line.split(",")
    if(temp[3]!=None):
        if(int(temp[3])>5 and int(temp[3])<5000):
            author_ids.append(temp[0])
            temp_url=url+'"'+temp[0]+'"'
            r=requests.get(temp_url).json()
            work_count+=len(r["results"])
    count+=1
    print(count)
print(work_count)

1
2
3
4
5


KeyboardInterrupt: 